In [1]:
import os
import random
import numpy as np

import tensorflow as tf
from tensorflow.keras.layers import *
from tensorflow.keras.models import *
from tensorflow.keras.callbacks import *
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator

from hyperopt import hp, Trials
from sklearn.model_selection import KFold

from kerashypetune import KerasBayesianSearch, KerasBayesianSearchCV

In [2]:
def set_seed(seed):
    
    tf.random.set_seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    random.seed(seed)

In [3]:
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()

np.random.seed(33)
subset_train = np.random.uniform(0,1, len(y_train))
subset_test = np.random.uniform(0,1, len(y_test))

x_train = x_train[subset_train < 0.1] / 255
y_train = y_train[subset_train < 0.1]

x_test = x_test[subset_test < 0.3] / 255
y_test = y_test[subset_test < 0.3]

x_train.shape, y_train.shape, x_test.shape, y_test.shape

((6036, 28, 28), (6036,), (2963, 28, 28), (2963,))

In [4]:
def get_model(param):
    
    set_seed(33)
        
    model = Sequential()
    model.add(Flatten())
    model.add(Dense(param['unit_1'], activation='relu'))
    model.add(Dense(param['unit_2'], activation='relu'))
    model.add(Dense(10, activation='softmax'))
    model.compile(optimizer=Adam(learning_rate=param['lr']), 
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])
    
    return model

In [5]:
param_grid = {
    'unit_1': 64 + hp.randint('unit_1', 64),
    'unit_2': 32 + hp.randint('unit_2', 96),
    'lr': hp.loguniform('lr', np.log(0.001), np.log(0.02)), 
    'epochs': 100,
    'batch_size': 512
}

# Fixed validation search

In [6]:
es = EarlyStopping(patience=10, verbose=1, min_delta=0.001, monitor='val_accuracy', mode='auto', restore_best_weights=True)

hypermodel = get_model  
# to pass external arguments to get_model use lambda function
# ==> hypermodel = lambda x: get_model(param=x, ...)
# callable(hypermodel) ==> True

kbs = KerasBayesianSearch(hypermodel, param_grid, n_iter=15, sampling_seed=33,
                          monitor='val_accuracy', greater_is_better=True, tuner_verbose=1)
kbs.search(x_train, y_train, trials=Trials(), validation_data=(x_test, y_test), callbacks=[es])


15 trials detected for ('unit_1', 'unit_2', 'lr', 'epochs', 'batch_size')

***** (1/15) *****
Search({'batch_size': 512, 'epochs': 100, 'lr': 0.0016861379017239324, 'unit_1': 122, 'unit_2': 58})
Restoring model weights from the end of the best epoch.
Epoch 00039: early stopping
SCORE: 0.9487 at epoch 29

***** (2/15) *****
Search({'batch_size': 512, 'epochs': 100, 'lr': 0.0014104783964893717, 'unit_1': 80, 'unit_2': 96})
Restoring model weights from the end of the best epoch.
Epoch 00037: early stopping
SCORE: 0.9433 at epoch 37

***** (3/15) *****
Search({'batch_size': 512, 'epochs': 100, 'lr': 0.006139711436597271, 'unit_1': 101, 'unit_2': 101})
Restoring model weights from the end of the best epoch.
Epoch 00036: early stopping
SCORE: 0.95005 at epoch 26

***** (4/15) *****
Search({'batch_size': 512, 'epochs': 100, 'lr': 0.018199685929694627, 'unit_1': 70, 'unit_2': 107})
Restoring model weights from the end of the best epoch.
Epoch 00020: early stopping
SCORE: 0.94566 at epoch 13



In [7]:
kbs.scores

[0.9487,
 0.9433,
 0.95005,
 0.94566,
 0.95376,
 0.94735,
 0.94904,
 0.95376,
 0.94398,
 0.95106,
 0.9514,
 0.9487,
 0.95343,
 0.946,
 0.95039]

In [8]:
kbs.best_score

0.95376

In [9]:
kbs.best_params

{'batch_size': 512,
 'epochs': 22,
 'lr': 0.0030185075402515246,
 'unit_1': 101,
 'unit_2': 112,
 'steps_per_epoch': 12}

In [10]:
kbs.best_model

# Fixed validation search Generator

In [11]:
gen = ImageDataGenerator(
         rotation_range=90,
         width_shift_range=0.1,
         height_shift_range=0.1,
         zoom_range=0.2)

In [12]:
es = EarlyStopping(patience=5, verbose=1, min_delta=0.001, monitor='val_accuracy', mode='auto', restore_best_weights=True)

hypermodel = get_model
# to pass external arguments to get_model use lambda function
# ==> hypermodel = lambda x: get_model(param=x, ...)
# callable(hypermodel) ==> True

kbs = KerasBayesianSearch(hypermodel, param_grid, n_iter=15, sampling_seed=33,
                          monitor='val_accuracy', greater_is_better=True, tuner_verbose=1)
kbs.search(gen.flow(np.expand_dims(x_train,-1), y_train, batch_size=param_grid['batch_size'], seed=33), 
           trials=Trials(),
           validation_data=(np.expand_dims(x_test,-1), y_test), 
           callbacks=[es], steps_per_epoch=len(x_train)//param_grid['batch_size'])


15 trials detected for ('unit_1', 'unit_2', 'lr', 'epochs', 'batch_size')

***** (1/15) *****
Search({'batch_size': 512, 'epochs': 100, 'lr': 0.0016861379017239324, 'unit_1': 122, 'unit_2': 58})
Restoring model weights from the end of the best epoch.
Epoch 00029: early stopping
SCORE: 0.8839 at epoch 24

***** (2/15) *****
Search({'batch_size': 512, 'epochs': 100, 'lr': 0.0014104783964893717, 'unit_1': 80, 'unit_2': 96})
Restoring model weights from the end of the best epoch.
Epoch 00023: early stopping
SCORE: 0.84914 at epoch 18

***** (3/15) *****
Search({'batch_size': 512, 'epochs': 100, 'lr': 0.006139711436597271, 'unit_1': 101, 'unit_2': 101})
Restoring model weights from the end of the best epoch.
Epoch 00018: early stopping
SCORE: 0.86196 at epoch 13

***** (4/15) *****
Search({'batch_size': 512, 'epochs': 100, 'lr': 0.018199685929694627, 'unit_1': 70, 'unit_2': 107})
Restoring model weights from the end of the best epoch.
Epoch 00027: early stopping
SCORE: 0.82923 at epoch 22


In [13]:
kbs.scores

[0.8839,
 0.84914,
 0.86196,
 0.82923,
 0.89976,
 0.88424,
 0.87951,
 0.87715,
 0.8758,
 0.88896,
 0.8866,
 0.8731,
 0.87006,
 0.87546,
 0.89571]

In [14]:
kbs.best_score

0.89976

In [15]:
kbs.best_params

{'batch_size': 512,
 'epochs': 32,
 'lr': 0.0030185075402515246,
 'unit_1': 101,
 'unit_2': 112,
 'steps_per_epoch': 11}

In [16]:
kbs.best_model

# Cross validation search

In [17]:
X = np.concatenate([x_train, x_test])
y = np.concatenate([y_train, y_test])

X.shape, y.shape

((8999, 28, 28), (8999,))

In [18]:
cv = KFold(n_splits=3, random_state=33, shuffle=True)

es = EarlyStopping(patience=10, verbose=1, min_delta=0.001, monitor='val_accuracy', mode='auto', restore_best_weights=True)

hypermodel = get_model
# to pass external arguments to get_model use lambda function
# ==> hypermodel = lambda x: get_model(param=x, ...)

kbs = KerasBayesianSearchCV(hypermodel, param_grid, cv=cv, n_iter=5, sampling_seed=33,
                            monitor='val_accuracy', greater_is_better=True, tuner_verbose=1)
kbs.search(X, y, trials=Trials(), callbacks=[es])


##################
###  Fold 001  ###
##################

5 trials detected for ('unit_1', 'unit_2', 'lr', 'epochs', 'batch_size')

***** (1/5) *****
Search({'batch_size': 512, 'epochs': 100, 'lr': 0.0016861379017239324, 'unit_1': 122, 'unit_2': 58})
Restoring model weights from the end of the best epoch.
Epoch 00028: early stopping
SCORE: 0.942 at epoch 28

***** (2/5) *****
Search({'batch_size': 512, 'epochs': 100, 'lr': 0.0014104783964893717, 'unit_1': 80, 'unit_2': 96})
Restoring model weights from the end of the best epoch.
Epoch 00030: early stopping
SCORE: 0.93633 at epoch 30

***** (3/5) *****
Search({'batch_size': 512, 'epochs': 100, 'lr': 0.006139711436597271, 'unit_1': 101, 'unit_2': 101})
Restoring model weights from the end of the best epoch.
Epoch 00024: early stopping
SCORE: 0.94633 at epoch 23

***** (4/5) *****
Search({'batch_size': 512, 'epochs': 100, 'lr': 0.018199685929694627, 'unit_1': 70, 'unit_2': 107})
Restoring model weights from the end of the best epoch.
Epo

In [19]:
kbs.folds_scores

{'fold 1': [0.942, 0.93633, 0.94633, 0.946, 0.94267],
 'fold 2': [0.951, 0.94567, 0.95233, 0.946, 0.95133],
 'fold 3': [0.94498, 0.94598, 0.95265, 0.94765, 0.95198]}

In [20]:
kbs.folds_best_score

{'fold 1': 0.94633, 'fold 2': 0.95233, 'fold 3': 0.95265}

In [21]:
kbs.folds_best_params

{'fold 1': {'batch_size': 512,
  'epochs': 23,
  'lr': 0.006139711436597271,
  'unit_1': 101,
  'unit_2': 101,
  'steps_per_epoch': 12},
 'fold 2': {'batch_size': 512,
  'epochs': 14,
  'lr': 0.006139711436597271,
  'unit_1': 101,
  'unit_2': 101,
  'steps_per_epoch': 12},
 'fold 3': {'batch_size': 512,
  'epochs': 16,
  'lr': 0.006139711436597271,
  'unit_1': 101,
  'unit_2': 101,
  'steps_per_epoch': 12}}

In [22]:
kbs.folds_best_models

{'fold 1': <tensorflow.python.keras.engine.sequential.Sequential at 0x1d8bd478dc8>,
 'fold 2': <tensorflow.python.keras.engine.sequential.Sequential at 0x1d8bd427dc8>,
 'fold 3': <tensorflow.python.keras.engine.sequential.Sequential at 0x1d8bd3fa808>}

In [23]:
kbs.best_params_score

0.95044

In [24]:
kbs.best_params

[{'batch_size': 512,
  'epochs': 23,
  'lr': 0.006139711436597271,
  'unit_1': 101,
  'unit_2': 101,
  'steps_per_epoch': 12},
 {'batch_size': 512,
  'epochs': 14,
  'lr': 0.006139711436597271,
  'unit_1': 101,
  'unit_2': 101,
  'steps_per_epoch': 12},
 {'batch_size': 512,
  'epochs': 16,
  'lr': 0.006139711436597271,
  'unit_1': 101,
  'unit_2': 101,
  'steps_per_epoch': 12}]